# CompSpec Network trained with Mini-batching

## Intro

* **Date**: 11/26/2020
* **What**: This is the competitive specializing network from [the last experiment](../11_24_2020_experiment_competitive_specializing_network), except trained with mini-batching.
* **Why**: The competitive specializing network blew my expectations out of the water, but it's still about 10x slower than [Krotov's and Hopfield's competitive algo](https://www.pnas.org/content/116/16/7723).  This algorithm converges typically with only one pass through the dataset, whereas Krotov's network takes like 100 passes.  If I can make this as fast as Krotov's network on a single pass through data, then bois, we'd be boolin'.
* **Hopes**: I want this algorithm to train incredibly quickly.  Is that too much to ask for?
* **Limitations**: I super don't know how to do mini-batching correctly.  Luckily, I have Dima's code to work off of, so hopefully it won't be a huge deal.

## Technicals

Basically, everything is the same as the algorithm I ended up with during the last experiment, but now I'm just trying to get it to work with mini-batches.  I'm not going to describe Mini-batching.  If you're reading this, you have an internet connection, so scoot-scoot your lil booty on over to Google and look it up.  It's incredibly straight forward.

## Code

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tqdm import tqdm

L = 28 * 28   #Size of mnist in pixels
S = 60000     #Size of training set

(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = train_X / 255.0

flat_x = np.reshape(train_X, [-1, 1, L])
flat_test = np.reshape(test_X, [-1, 1, L])

In [3]:
def draw_weights(synapses, Kx, Ky): 
    yy=0
    HM=np.zeros((28*Ky,28*Kx))
    for y in range(Ky):
        for x in range(Kx):
            HM[y*28:(y+1)*28,x*28:(x+1)*28]=synapses[yy,:].reshape(28,28)
            yy += 1
    plt.clf()
    nc=np.amax(np.absolute(HM))
    
    im=plt.imshow(HM,cmap='Greys',vmin=0,vmax=nc)
    fig.colorbar(im,ticks=[0, np.amax(HM)])
    plt.axis('off')
    fig.canvas.draw()

In [4]:
def wta_classification(w, T_s):
    flat_x = np.reshape(train_X, [-1, L])
    flat_test = np.reshape(test_X, [-1, L])
    
    v = flat_x[:T_s]
    train_lbls = train_y[:T_s]
    
    v = v / np.array([np.linalg.norm(v, axis=1)]).T
    w = w / np.array([np.linalg.norm(w, axis=1)]).T
    
    wins = np.argmax(w @ (flat_x[:T_s, :]).T, axis=0)
    
    n_wins = np.zeros((w.shape[0], 10))
    
    for (n_i, lbl) in zip(wins, train_lbls):
        n_wins[n_i][lbl] += 1
        
    n_cls = np.argmax(n_wins, axis=1)
    print("Neuron classes:", n_cls)
    
    num_test = test_y.shape[0]
    test_v = flat_test / np.array([np.linalg.norm(flat_test, axis=1)]).T
    
    num_correct = 0
    
    incorrect = np.zeros(10)
    
    for i in range(num_test):
        w_mul_v = w @ test_v[i].T
        
        n_max = np.argmax(w_mul_v)
        
        pre = n_cls[n_max]
        if pre == test_y[i]:
            num_correct += 1
        else:
            incorrect[test_y[i]] += 1
            
    print("Accuracy:", str(num_correct * 100 / num_test) + '%')
    print("Misclassifieds: ", incorrect)
        